In [ ]:
import nibabel as nib
import numpy as np
import glob
import matplotlib.pyplot as plt

path_tumors = '/mnt/data/psteinmetz/neotex/to_process/Ozgun/*/RoiVolume/VOI.nii.gz'
tumor_mask_files = glob.glob(path_tumors)

for tumor_mask_file in tumor_mask_files:
    # Load the tumor mask nifti file
    tumor_mask_nii = nib.load(tumor_mask_file)
    tumor_mask_data = tumor_mask_nii.get_fdata()

    # Find the bounding box around the tumor
    non_zero_coords = np.argwhere(tumor_mask_data > 0) # Find all non-zero points in the mask
    min_coords = non_zero_coords.min(axis=0)  # Minimum corner of the bounding box
    max_coords = non_zero_coords.max(axis=0)  # Maximum corner of the bounding box
    
    # Debugging: Print the bounding box coordinates and tumor mask shape
    print(f"Tumor mask shape: {tumor_mask_data.shape}")
    print(f"Bounding box min coords: {min_coords}")
    print(f"Bounding box max coords: {max_coords}")

    # Create a new empty mask of the same shape as the original
    bounding_box_mask = np.zeros_like(tumor_mask_data)

    # Set the bounding box region to 1 (or another value if needed)
    bounding_box_mask[min_coords[0]:max_coords[0]+1,
                    min_coords[1]:max_coords[1]+1,
                    min_coords[2]:max_coords[2]+1] = 1
    
    # Debugging: Check if the bounding box was applied correctly
    bounding_box_voxels = np.argwhere(bounding_box_mask == 1)
    print(f"Number of bounding box voxels: {bounding_box_voxels.shape[0]}")
    print(f"Unique values in bounding box mask: {np.unique(bounding_box_mask)}")

    # Create a new NIfTI image from the bounding box mask
    bounding_box_nii = nib.Nifti1Image(bounding_box_mask, tumor_mask_nii.affine, tumor_mask_nii.header)

    # Save the new bounding box mask
    bounding_box_file = tumor_mask_file[:-7] + '_bb.nii.gz'
    nib.save(bounding_box_nii, bounding_box_file)

    print(f"Bounding box mask saved to: {bounding_box_file}")

